In [214]:
import pandas as pd
import numpy as np
import joblib

In [215]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [216]:
#df = pd.read_csv('data_by_location/Kasurila_data.csv') # Error 8.88
#df = pd.read_csv('data_by_location/Levi_data.csv') # Error 17.05
#df = pd.read_csv('data_by_location/Luosto_data.csv') # Error 13.97
#df = pd.read_csv('data_by_location/Messila_data.csv') # Error 6.61
#df = pd.read_csv('data_by_location/Mustavaara_data.csv') # Error 9.38
#df = pd.read_csv('data_by_location/Ounasvaara_data.csv') # Error 12.27
#df = pd.read_csv('data_by_location/Purnu_data.csv') # Error 8.10
#df = pd.read_csv('data_by_location/Ruka_data.csv') # Error 11.46
#df = pd.read_csv('data_by_location/Ruunarinteet_data.csv') # Error 7.28
#df = pd.read_csv('data_by_location/Salla_data.csv') # Error 13.26

In [217]:
# Step 1: Enhance your data preprocessing to extract temporal features
def preprocess_with_time_features(file_path):
    df = pd.read_csv(file_path)
    
    # Remove duplicates
    df = df.drop_duplicates(subset=['date'], keep='last')
    
    # Handle missing values
    df.replace(to_replace='-', value=np.nan, inplace=True)
    df.snow_depth_cm = df.snow_depth_cm.replace(to_replace='-1', value='0')
    
    # Convert to numeric
    df['avg_temp_c'] = pd.to_numeric(df['avg_temp_c'], errors='coerce')
    df['snow_depth_cm'] = pd.to_numeric(df['snow_depth_cm'], errors='coerce')
    df['uv_index'] = pd.to_numeric(df['uv_index'], errors='coerce')
    
    # Convert date string to datetime object
    df['date'] = pd.to_datetime(df['date'])
    
    # Extract time-based features
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_year'] = df['date'].dt.dayofyear
    
    # Drop rows with missing target values
    df = df.dropna(subset=['snow_depth_cm'])
    
    return df

# Step 2: Combine datasets with time features
datasets = [
    'data_by_location/Messila_data.csv',
    'data_by_location/Purnu_data.csv',
    'data_by_location/Ruunarinteet_data.csv',
    'data_by_location/Kasurila_data.csv'
]

all_data = []
for dataset in datasets:
    try:
        df = preprocess_with_time_features(dataset)
        all_data.append(df)
    except Exception as e:
        print(f"Error processing {dataset}: {e}")

combined_df = pd.concat(all_data)

# Step 3: Create features including time components
X = combined_df[['avg_temp_c', 'uv_index', 'cloud_cover_rate', 'month', 'day_of_year']]
y = combined_df['snow_depth_cm']

Error processing data_by_location/Purnu_data.csv: 'uv_index'


In [218]:
df.columns

Index(['date', 'snow_depth_cm', 'avg_temp_c', 'uv_index', 'cloud_cover_rate',
       'cloud_cover', 'location', 'year', 'month', 'day', 'day_of_year'],
      dtype='object')

In [219]:
df.sample(15)

,date,snow_depth_cm,avg_temp_c,uv_index,cloud_cover_rate,cloud_cover,location,year,month,day,day_of_year
2018,2008-09-17,0.0,6.9,0.2,7.0,Mostly cloudy,Kasurila,2008,9,17,261
6771,2021-09-22,0.0,4.3,0.3,0.0,Clear,Kasurila,2021,9,22,265
6003,2019-08-16,0.0,16.1,0.4,8.0,Cloudy,Kasurila,2019,8,16,228
2040,2008-10-09,0.0,2.6,0.2,0.0,Clear,Kasurila,2008,10,9,283
4633,2015-11-15,0.0,1.2,0.0,7.0,Mostly cloudy,Kasurila,2015,11,15,319
4213,2014-09-21,0.0,12.2,0.3,8.0,Cloudy,Kasurila,2014,9,21,264
4449,2015-05-15,0.0,6.6,0.7,3.0,Mostly clear,Kasurila,2015,5,15,135
3438,2012-08-07,0.0,17.6,0.2,1.0,Clear,Kasurila,2012,8,7,220
3937,2013-12-19,8.0,-1.5,0.0,0.0,Clear,Kasurila,2013,12,19,353
3519,2012-10-27,0.0,-2.9,0.0,7.0,Mostly cloudy,Kasurila,2012,10,27,301


In [220]:
df.cloud_cover.value_counts()

cloud_cover
Clear                              2370
Mostly cloudy                      2097
Cloudy                             1864
Partly cloudy                       355
Mostly clear                        276
Cloudiness cannot be determined      76
Name: count, dtype: int64

In [221]:
df.replace(to_replace='-', value=np.nan, inplace=True)

In [222]:
df.snow_depth_cm = df.snow_depth_cm.replace(to_replace='-1', value='0')

In [223]:
df.cloud_cover.value_counts()

cloud_cover
Clear                              2370
Mostly cloudy                      2097
Cloudy                             1864
Partly cloudy                       355
Mostly clear                        276
Cloudiness cannot be determined      76
Name: count, dtype: int64

In [224]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [225]:
df.columns

Index(['date', 'snow_depth_cm', 'avg_temp_c', 'uv_index', 'cloud_cover_rate',
       'cloud_cover', 'location', 'year', 'month', 'day', 'day_of_year'],
      dtype='object')

In [226]:
df.dtypes

date                datetime64[ns]
snow_depth_cm              float64
avg_temp_c                 float64
uv_index                   float64
cloud_cover_rate           float64
cloud_cover                 object
location                    object
year                         int32
month                        int32
day                          int32
day_of_year                  int32
dtype: object

In [227]:
df.avg_temp_c.value_counts()

avg_temp_c
 0.5     53
 1.7     49
 0.7     49
 0.3     45
 0.8     44
         ..
-23.1     1
-28.0     1
-22.1     1
 25.8     1
-20.5     1
Name: count, Length: 520, dtype: int64

In [228]:
df.snow_depth_cm.value_counts()

snow_depth_cm
0.0     4198
2.0       93
3.0       84
4.0       78
5.0       70
        ... 
85.0       1
84.0       1
95.0       1
93.0       1
91.0       1
Name: count, Length: 90, dtype: int64

In [229]:
# Remove duplicates
combined_df = combined_df.drop_duplicates(subset=['date'], keep='last')

In [230]:
combined_df['avg_temp_c'] = pd.to_numeric(combined_df['avg_temp_c'], errors='coerce')

In [231]:
combined_df['uv_index'] = pd.to_numeric(combined_df['uv_index'], errors='coerce')

In [232]:
combined_df = combined_df.dropna(subset=['avg_temp_c'])
combined_df = combined_df.dropna(subset=['snow_depth_cm'])

In [233]:
# Only values after 2006
selector = combined_df['date'] >= '2006-01-01'
combined_df = combined_df[combined_df['date'] >= '2006-01-01']

In [234]:
# Create lag features

combined_df['snow_depth_1d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(1)

combined_df['snow_depth_7d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(7)

combined_df['snow_depth_365d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(365)
 

In [235]:
# Convert date string to datetime object
combined_df['date'] = pd.to_datetime(combined_df['date'])
    
# Extract time-based features
combined_df['year'] = combined_df['date'].dt.year
combined_df['month'] = combined_df['date'].dt.month
combined_df['day'] = combined_df['date'].dt.day
combined_df['day_of_year'] = combined_df['date'].dt.dayofyear

In [236]:
# Create an XGBoost regressor model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

X = combined_df[['avg_temp_c', 'cloud_cover_rate','uv_index', 'snow_depth_1d_ago', 'snow_depth_7d_ago', 'snow_depth_365d_ago', 'year', 'month', 'day', 'day_of_year']]  # Features 
y = combined_df['snow_depth_cm']  # Target variable (snow depth)

# Split into training (80%) and test (20%) data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

joblib.dump(model, 'snow_depth_time_model.joblib')

['snow_depth_time_model.joblib']

In [237]:
predictions = model.predict(X_test)

In [238]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error: {mae}")

mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Absolute Error: 0.7622424854641255
Mean Squared Error: 2.729005712950477


In [239]:
# Step 5: Function to predict future years
def predict_future_snow_depth(model, start_date, days=3650, location_data=None):
    """
    Predict snow depth for future dates
    
    Parameters:
    - model: Trained model
    - start_date: Start date for predictions (string YYYY-MM-DD or datetime)
    - days: Number of days to predict forward
    - location_data: Sample data from a location to use as base values
    
    Returns:
    - DataFrame with dates and predicted snow depths
    """
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    
    # Create date range
    future_dates = [start_date + timedelta(days=i) for i in range(days)]
    future_df = pd.DataFrame({'date': future_dates})
    # Create lag features

    future_df['snow_depth_1d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(1)

    future_df['snow_depth_7d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(7)

    future_df['snow_depth_365d_ago'] = combined_df.groupby('location')['snow_depth_cm'].shift(365)
    
    # Extract time features
    future_df['year'] = future_df['date'].dt.year
    future_df['month'] = future_df['date'].dt.month
    future_df['day'] = future_df['date'].dt.day
    future_df['day_of_year'] = future_df['date'].dt.dayofyear
    
    # Generate weather features based on historical averages by day of year
    if location_data is not None:
        # Group location data by day of year and get averages
        daily_averages = location_data.groupby('day_of_year').agg({
            'avg_temp_c': 'mean',
            'uv_index': 'mean',
            'cloud_cover_rate': 'mean'
        }).reset_index()
        
        # Merge with future dates
        future_df = future_df.merge(daily_averages, on='day_of_year', how='left')
        
        # For days not in historical data, use nearest day
        future_df = future_df.fillna(method='ffill').fillna(method='bfill')
    else:
        # If no location data is provided, use seasonal patterns
        # This is simplified - would be better with actual historical weather data
        future_df['month_rad'] = future_df['month'] * 2 * np.pi / 12
        
        # Temperature follows seasonal cycle (simplified model)
        # Northern hemisphere: coldest in Jan/Feb, warmest in Jul/Aug
        future_df['avg_temp_c'] = -10 * np.cos(future_df['month_rad']) + 5
        
        # UV follows similar seasonal pattern
        future_df['uv_index'] = 3 * np.cos((future_df['month_rad'] + np.pi)) + 3
        
        # Cloud cover (simplified)
        future_df['cloud_cover_rate'] = 0.5 + 0.2 * np.sin(future_df['month_rad'])
    
    # Prepare features for prediction in the same format as training data
    X_future = future_df[['avg_temp_c', 'cloud_cover_rate','uv_index', 'snow_depth_1d_ago', 'snow_depth_7d_ago', 'snow_depth_365d_ago', 'year', 'month', 'day', 'day_of_year']]
    
    # Make predictions
    future_df['predicted_snow_depth'] = model.predict(X_future)
    
    # Ensure non-negative snow depths
    future_df['predicted_snow_depth'] = future_df['predicted_snow_depth'].clip(lower=0)
    
    return future_df[['date', 'predicted_snow_depth', 'avg_temp_c', 'uv_index', 'cloud_cover_rate']]

# Example usage:
# 1. Load a sample location dataset for reference weather patterns
sample_location = preprocess_with_time_features('data_by_location/Messila_data.csv')

# 2. Load the trained model
trained_model = joblib.load('snow_depth_time_model.joblib')

# 3. Predict snow depth for next year starting from a specific date
start_prediction_date = '2025-01-01'
future_predictions = predict_future_snow_depth(
    model=trained_model,
    start_date=start_prediction_date,
    days=3650,
    location_data=sample_location
)

C:\Users\KaisaVuori\AppData\Local\Temp\ipykernel_31300\1011408554.py:48: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future_df = future_df.fillna(method='ffill').fillna(method='bfill')


In [240]:
future_predictions.to_csv('snow_depth_predictions_2025.csv', index=False)
print("Future predictions saved to snow_depth_predictions_2025.csv")

Future predictions saved to snow_depth_predictions_2025.csv
